## Overview
## Outline
1. Import Library
2. Setup pre-requisites
3. Extract dataset to images
4. Upload images to Azure data store
4. Setting up Azure ML Infrastructure

### 1. Import library

In [ ]:
## Install needed packages
%pip install opencv-python torch torchvision torchaudio azureml-core scikit-learn tf2onnx onnx2pytorch

In [ ]:
from onnx2pytorch import ConvertModel
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
import tf2onnx

model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Conv2D(96, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(24, activation='softmax')
])
# Convert the model to ONNX format
onnx_model, _ = tf2onnx.convert.from_keras(model)

ConvertModel(onnx_model)

In [2]:
import os
from importlib import reload

import cv2
# from azureml.core import Workspace, Dataset, Datastore

## Using torchvision to create a dataset
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torchvision
import torch


### 2. Setup pre-requisites

Clone source code from github

In [ ]:
## curl source code from github
!wget 'https://github.com/nnvu-fit/iusai-project/archive/refs/heads/main.zip' -O main.zip
!unzip main.zip
!mv iusai-project-main/* .
!rm -rf iusai-project-main main.zip

In [ ]:
## Setup Azure ML Workspace
ws = Workspace.from_config()
## From workspace, get/create the default datastore
ds = ws.get_default_datastore()
ws, ds

### 3. Extract dataset to images

In [ ]:
## define videos location + images output location
video_path = 'videos/'
images_path = 'subjects-small/'
print('video_path: ', video_path)
print('images_path: ', images_path)

In [ ]:
import v2i
reload(v2i)
from v2i import extract_images_from_videos
import threading

def extract_images_from_videos_collection(video_path, images_path):
    ## get all videos file in video_path
    video_list_location_collection = os.listdir(video_path)

    ## define total_label_dict
    ## create threading pool
    thread = []
    ## for each video file
    for video_list_location in video_list_location_collection:
        ## check if video_location is not a directory (i.e. is a file), then skip
        if not os.path.isdir(video_path + video_list_location):
            continue
        ## list videos in video_location
        thread.append(threading.Thread(target=extract_images_from_videos, args=(video_path + video_list_location, images_path, 1)))
        # extract_images_from_videos(video_path + video_list_location, images_path, inteval=1)

    ## start all threads
    for t in thread: t.start()
    ## wait for all threads to finish
    for t in thread:
        t.join()
    
    ## check images in images_path
    images_list_location_collection = os.listdir(images_path)
    print('images_list_location_collection: ', images_list_location_collection)

# video_subject_6_path = video_path + '/subject6-###'
# ## list videos in video_location
# label_dict = extract_images_from_videos(video_subject_6_path, images_path)

## extract images from videos
extract_images_from_videos_collection(video_path, images_path)


### 4. Upload images to Azure data store

In [ ]:
## upload images to data asset using Dataset.File.upload_directory
Dataset.File.upload_directory(src_dir=images_path, target=(ds, 'images-extra-small'), overwrite=True, show_progress=True) \
    .register(workspace=ws, name='images-extra-small', description='images-extra-small') ## register dataset \\


### 5. Setup public workspace endpoint

In [ ]:
dataset_collection_list = ['images-extra-small', 'images-small-v2', 'images-medium']
## register dataset using Dataset.File.from_files
for dataset_collection in dataset_collection_list:
    ## check if dataset_collection is already registered
    if dataset_collection in ws.datasets.keys():
        print('dataset_collection: ', dataset_collection, ' is already registered')
        continue
    Dataset.File.from_files(path=(ds, dataset_collection)) \
        .register(workspace=ws, name=dataset_collection, description=dataset_collection) ## register dataset \\


## Define and Train models
1. ResNET
2. DenseNET

### Setup device + load dataset

In [ ]:
images_path = 'images'
images_ds_path = "images-extra-small"
if not os.path.exists(images_path):
    os.mkdir(images_path)
# download data asset to local if images_path is empty
if len(os.listdir(images_path)) == 0:
    print('images_path is empty, download images_ds to images_path')
    # download data asset to local
    images_ds = Dataset.get_by_name(workspace=ws, name=images_ds_path)
    images_ds.download(target_path=images_path, overwrite=True)

#### Prepare Dataset

In [20]:
from torch.utils.data import DataLoader
from torchvision import transforms

transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
imageDataset = ds.ImageDataset('datasets/subjects-small', transform=transform)
## define batch_size
batch_size = 32

In [21]:
image, label = imageDataset.get_image(0)
# image.show()
labels = imageDataset.labels()
## show labels in Interger
print('labels: ',  labels)

labels:  ['subject1', 'subject10', 'subject11', 'subject12', 'subject13', 'subject14', 'subject15', 'subject16', 'subject17', 'subject18', 'subject19', 'subject2', 'subject20', 'subject21', 'subject22', 'subject23', 'subject24', 'subject3', 'subject4', 'subject5', 'subject6', 'subject7', 'subject8', 'subject9']


In [23]:
## split dataset into train and test dataset using random_split
from torch.utils.data import random_split
train_val_size = int(0.9 * len(imageDataset))
test_size = len(imageDataset) - train_val_size
train_val_ds, test_ds = random_split(imageDataset, [train_val_size, test_size])

## split train_val_ds into train_ds and val_ds using random_split
train_size = int(0.8 * len(train_val_ds))
val_size = len(train_val_ds) - train_size

train_ds, val_ds = random_split(train_val_ds, [train_size, val_size])

print('train_val_ds: ', len(train_val_ds))
print('test_ds: ', len(test_ds))


train_val_ds:  22637
test_ds:  2516


### 1. Resnes18

In [26]:
## import train and scrore function from train.py
import train as t
reload(t)
from train import ClassifierTrainer

## get resnet model of image classification from torchvision
model = torchvision.models.resnet18(pretrained=True)
## define optimizer using Adam and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer, loss_fn

trainer = ClassifierTrainer(model, optimizer, loss_fn, random_seed_value=86)

print('device: ', trainer.device)

In [ ]:
## train model
avg_lost = trainer.cross_validate(train_val_ds, epochs=5)
print('avg_lost: ', avg_lost)
## score model
test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)
trainer.score(test_dataloader)

In [ ]:
## save model
torch.save(model.state_dict(), 'model_resnes18.100_epochs.small.pth')

### 1. Resnes34

In [ ]:
## get resnet model of image classification from torchvision
model = torchvision.models.resnet34(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1)
## define optimizer using Adam and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer, loss_fn

In [10]:
## train model using device (CPU or GPU) 10 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=1, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

In [ ]:
## save model
torch.save(model.state_dict(), 'model_resnes34.small.pth')

### 2. DenseNet

In [ ]:
## get resnet model of image classification from torchvision
model = torchvision.models.densenet121(pretrained=True)
## define optimizer using Adam and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer, loss_fn

In [ ]:
## train model
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=1, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

In [ ]:
## train model using device (CPU or GPU) 10 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=100, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

In [ ]:
## save model
torch.save(model.state_dict(), 'model_resnes18.100_epochs.small.pth')

In [ ]:
## get denseNet model of image classification from torchvision
model = torchvision.models.densenet121(pretrained=True, num_classes=len(labels))
## define optimizer using Adam and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer, loss_fn

In [ ]:
## train model using device (CPU or GPU) 1 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=1, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

In [ ]:
## train model using device (CPU or GPU) 10 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=10, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

In [23]:
## save model
torch.save(model.state_dict(), 'model_densenet121.small.pth')

### 3. Using Yolov5 to extract face then using ResNet to classification the output

In [ ]:
## use yoloresnet model
import model
reload(model)
from model import YoloResnet
model = YoloResnet()
## define optimizer using Adam and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer, loss_fn

In [ ]:
## train model using device (CPU or GPU) 1 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=1, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

In [ ]:
## train model using device (CPU or GPU) 10 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=10, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

In [ ]:
## save model
torch.save(model.state_dict(), 'model_yoloresnet.pth')

In [2]:
import cv2

## load opencv haarcascade eye from xml file
haarcasecade_eye_xml = './opencv/haarcascade_eye.xml'
eye_cascade = cv2.CascadeClassifier(haarcasecade_eye_xml)

## open camera
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    ## show frame with mirror effect
    frame = cv2.flip(frame, 1)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    ## convert frame to gray
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ## detect eyes in frame
    eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in eyes:
        ## draw rectangle around eyes
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    ## show frame
    cv2.imshow('frame', frame)

cap.release()
cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np

## load opencv haarcascade eye from xml file
haarcasecade_eye_xml = './opencv/haarcascade_eye.xml'
eye_cascade = cv2.CascadeClassifier(haarcasecade_eye_xml)

## load opencv haarcascade face from xml file
haarcasecade_face_xml = './opencv/haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haarcasecade_face_xml)

## open camera
cap = cv2.VideoCapture(0)
try:
    while True:
        ret, frame = cap.read()
        ## show frame with mirror effect
        frame = cv2.flip(frame, 1)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        ## convert frame to gray
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ## detect faces in frame
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        for (x, y, w, h) in faces:
            ## draw rectangle around faces
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            ## crop face from frame
            face = frame[y:y+h, x:x+w]
            ## convert face to gray
            gray_face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            ## detect eyes in face
            eyes = eye_cascade.detectMultiScale(gray_face, 1.1, 4)
            for (ex, ey, ew, eh) in eyes:
                ## draw rectangle around eyes
                cv2.rectangle(face, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)


        ## show frame
        cv2.imshow('frame', frame)
finally:
    cap.release()
    cv2.destroyAllWindows()



In [20]:
import cv2
import torch
import torchvision
from torchvision import transforms

# load model faster_rcnn_resnet50_fpn from torchvision
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    weights=torchvision.models.detection.FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

# using cuda if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

try:
    ## open camera
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        ## show frame with mirror effect
        frame = cv2.flip(frame, 1)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        ## convert frame to tensor
        image = transforms.ToTensor()(frame)
        ## add batch dimension
        image = image.unsqueeze(0)
        ## move image to device
        image = image.to(device)

        ## detect eyes
        model.eval()
        with torch.no_grad():
            prediction = model(image)

        ## get prediction boxes
        boxes = prediction[0]['boxes']
        ## get prediction labels
        labels = prediction[0]['labels']

        ## only get the boxes with face label
        face_label = 1
        boxes = boxes[labels == face_label]
        labels = labels[labels == face_label]

        ## draw boxes on image
        for box, label in zip(boxes, labels):
            box = box.int()
            label = label.item()

            ## turn box to list
            box = box.tolist()

            x_start = box[0], box[1]
            x_end = box[2], box[3]
            
            ## draw rectangle around eyes
            cv2.rectangle(frame, x_start, x_end, (255, 0, 0), 2)

        ## show frame
        cv2.imshow('frame', frame)
finally:
    cap.release()
    cv2.destroyAllWindows()


box:  tensor([173, 196, 470, 468], device='cuda:0', dtype=torch.int32)
label:  1
box:  tensor([106,  54, 540, 480], device='cuda:0', dtype=torch.int32)
label:  1
box:  tensor([347, 196, 526, 476], device='cuda:0', dtype=torch.int32)
label:  1
box:  tensor([190, 195, 483, 467], device='cuda:0', dtype=torch.int32)
label:  1
box:  tensor([322, 196, 522, 477], device='cuda:0', dtype=torch.int32)
label:  1
box:  tensor([277, 138, 312, 182], device='cuda:0', dtype=torch.int32)
label:  1
box:  tensor([196, 196, 481, 471], device='cuda:0', dtype=torch.int32)
label:  1
box:  tensor([132,  41, 552, 480], device='cuda:0', dtype=torch.int32)
label:  1
box:  tensor([ 71, 195, 390, 472], device='cuda:0', dtype=torch.int32)
label:  1
box:  tensor([316, 168, 523, 474], device='cuda:0', dtype=torch.int32)
label:  1
box:  tensor([277, 138, 311, 182], device='cuda:0', dtype=torch.int32)
label:  1
box:  tensor([193, 195, 482, 466], device='cuda:0', dtype=torch.int32)
label:  1
box:  tensor([132,  77, 543,

KeyboardInterrupt: 

In [23]:
from importlib import reload
import train
reload(train)
from train import get_device

get_device()

'cuda'